Tôi sẽ giúp bạn chỉnh sửa `ChatbotWidget.tsx` một cách an toàn để tránh gửi tin nhắn trùng lặp. Đây là phần cần sửa:

```typescript:web/app/components/mindpal/chatbot/ChatbotWidget.tsx
const handleQuestion = async (event: Event) => {
  const questionEvent = event as SendQuestionEvent;
  const { question, context } = questionEvent.detail;

  if (window.chatbot?.sendMessage) {
    const message = `${context}\n\n${question}`;
    try {
      window.chatbot.sendMessage(message);
    } catch (error) {
      console.error('Error sending message:', error);
    }
  } else {
    console.error('Chatbot not ready');
  }
};
```

Thực ra, sau khi xem xét kỹ lại code, chúng ta **nên giữ lại** đoạn `window.chatbot.sendMessage(message)` trong `ChatbotWidget.tsx` vì:

1. Trong `RecommendQ.tsx`, chúng ta chỉ dispatch event `sendQuestion`
2. `ChatbotWidget.tsx` là nơi duy nhất xử lý event này và gửi tin nhắn thực sự đến chatbot
3. Nếu bỏ đoạn code này đi, tin nhắn sẽ không được gửi đến chatbot

Vấn đề gửi tin nhắn 2 lần đã được giải quyết trong `RecommendQ.tsx` bằng cách:
- Đã comment out phần gọi trực tiếp `window.chatbot.sendMessage`
- Chuyển sang sử dụng event system

Do đó, không cần phải bỏ đoạn code này trong `ChatbotWidget.tsx`. Nó đang hoạt động đúng như mong muốn.


## lý thuyết là thế, nhưng lúc chạy code lại bị duplicate

Nếu vẫn bị duplicate, hãy kiểm tra và sửa lại như sau:

```typescript:web/app/components/mindpal/chatbot/ChatbotWidget.tsx
const handleQuestion = async (event: Event) => {
  const questionEvent = event as SendQuestionEvent;
  const { question, context } = questionEvent.detail;

  if (window.chatbot?.sendMessage) {
    const message = `${context}\n\n${question}`;
    // Bỏ phần gửi tin nhắn ở đây vì có thể window.chatbot đã tự xử lý event
    // try {
    //   window.chatbot.sendMessage(message);
    // } catch (error) {
    //   console.error('Error sending message:', error);
    // }
  } else {
    console.error('Chatbot not ready');
  }
};
```

Lý do chỉnh sửa:
1. Có thể `window.chatbot` đã có event listener riêng xử lý `sendQuestion`
2. Việc chúng ta lắng nghe event và gọi lại `sendMessage` có thể tạo ra duplicate
3. Ta vẫn giữ phần kiểm tra `window.chatbot?.sendMessage` để log lỗi nếu chatbot chưa sẵn sàng

Hãy thử chạy lại và kiểm tra xem còn bị duplicate không. Nếu vẫn có vấn đề, chúng ta có thể cần kiểm tra thêm các event listener khác trong hệ thống.




Tôi thấy vấn đề duplicate message đến từ file `embed.js`. Trong hàm `sendMessage`, chúng ta đang gửi message 2 lần. Hãy sửa lại như sau:

```javascript:frontend/embed.js
window.chatbot = {
    sendMessage: async (message) => {
        // Hiển thị chatbot nếu đang ẩn
        iframe.contentWindow.postMessage({ type: 'showChatbot' }, '*');
        iframe.style.display = 'block';
        toggleButton.style.display = 'none';
        
        // Đợi một chút để giao diện cập nhật
        await new Promise(resolve => setTimeout(resolve, 100));
        
        // Chỉ gửi tin nhắn một lần
        iframe.contentWindow.postMessage({ type: 'sendMessage', message }, '*');
    },

    showChat: () => {
        iframe.contentWindow.postMessage({ type: 'showChatbot' }, '*');
        iframe.style.display = 'block';
        toggleButton.style.display = 'none';
    }
};
```

Những thay đổi chính:
1. Bỏ đi lần gọi `postMessage` thứ hai trong hàm `sendMessage`
2. Giữ nguyên flow: hiển thị chatbot -> đợi giao diện cập nhật -> gửi tin nhắn
3. Giữ nguyên các chức năng khác của chatbot

Điều này sẽ đảm bảo mỗi tin nhắn chỉ được gửi một lần.


```js
        sendMessage: async (message) => {
            iframe.contentWindow.postMessage({ type: 'showChatbot' }, '*'); // Thêm dòng này
            // vấn đề duplicate của message --- ra là 2 lần dòng code này.
            // iframe.contentWindow.postMessage({ type: 'sendMessage', message }, '*');            
            // Đảm bảo chat được mở trước
            iframe.style.display = 'block';
            toggleButton.style.display = 'none';
            
            // Đợi một chút để giao diện cập nhật
            await new Promise(resolve => setTimeout(resolve, 100));
            
            // Gửi tin nhắn
            iframe.contentWindow.postMessage({ type: 'sendMessage', message }, '*');
        },
```